In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
import pydataset 
from sklearn.feature_selection import f_regression, SelectKBest, RFE
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
# 1. Load the Tips dataset
# a. Create a price_per_person column
tips = pydataset.data('tips')
tips['price_per_person'] = tips.total_bill / tips.size
tips.head()

,total_bill,tip,sex,smoker,day,time,size,price_per_person
1,16.99,1.01,Female,No,Sun,Dinner,2,0.009947
2,10.34,1.66,Male,No,Sun,Dinner,3,0.006054
3,21.01,3.50,Male,No,Sun,Dinner,3,0.012301
4,23.68,3.31,Male,No,Sun,Dinner,2,0.013864
5,24.59,3.61,Female,No,Sun,Dinner,4,0.014397


In [5]:
#b. Before using any of the methods discussed in the lesson, 
#which features do you think would be most important for predicting the tip amount?

x = tips[['size', 'total_bill', 'price_per_person']]
y = tips.tip

In [6]:
# Calculate tip percentage. DON'T USE FOR PREDICTION, ONLY ANALYSIS
tips['tip_percentage'] = tips.tip / tips.total_bill

In [11]:
# c. Use select k best to select the top 2 features for predicting tip amount. What are they?

kbest = SelectKBest(f_regression, k=2)
kbest.fit(x, y)
print('Top two features according to k-best:')
x.columns[kbest.get_support()]

Top two features according to k-best:


Index(['total_bill', 'price_per_person'], dtype='object')

In [13]:
# d. Use recursive feature elimination to select the top 2 features for tip amount. What are they?

model = LinearRegression()
rfe = RFE(model, n_features_to_select=2).fit(x, y)
print('Top two features according to RFE:')
x.columns[rfe.get_support()]

Top two features according to RFE:


Index(['size', 'total_bill'], dtype='object')

In [34]:
# e. Why do you think select k best and recursive feature elimination might give different answers for the top features? 
# Does this change as you change the number of features your are selecting?

print('Select K_best is working with the best correlation with the target variable (y). RFE is based on the performance of a linear regression model')

Select K_best is working with the best correlation with the target variable (y). RFE is based on the performance of a linear regression model


#### 2 Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [14]:
def select_kbest(x, y, k):
    kbest = SelectKBest(k=k)
    kbest.fit(x, y)
    mask = kbest.get_support()
    return x.columns[mask]

In [15]:
select_kbest(tips[['total_bill', 'price_per_person', 'size']], tips.tip, 2)

Index(['total_bill', 'price_per_person'], dtype='object')

#### 3. Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [16]:
def rfe(X, y, k, model=LinearRegression()):
    rfe = RFE(model, n_features_to_select=k)
    rfe.fit(X, y)
    mask = rfe.get_support()
    return X.columns[mask]

In [17]:
rfe(tips[['total_bill', 'price_per_person', 'size']], tips.tip, 2)

Index(['total_bill', 'size'], dtype='object')

In [20]:
rfe(tips[['total_bill', 'price_per_person', 'size']], tips.day, 2, model=LogisticRegression())

/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-re

Index(['total_bill', 'size'], dtype='object')

#### 4. Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [26]:
swiss = pydataset.data('swiss')
swiss.columns

Index(['Fertility', 'Agriculture', 'Examination', 'Education', 'Catholic',
       'Infant.Mortality'],
      dtype='object')

In [24]:
x = swiss.drop(columns='Fertility')
y = swiss.Fertility

In [25]:
#K best first
select_kbest(x,y,3)

Index(['Agriculture', 'Examination', 'Education'], dtype='object')

In [27]:
# Recursive feature elimination 

rfe(x,y,3)

Index(['Examination', 'Education', 'Infant.Mortality'], dtype='object')

In [28]:
from sklearn.metrics import mean_squared_error

In [31]:
kbest_cols = select_kbest(x, y, 3)
x_kbest = x[kbest_cols]

model_kbest = LinearRegression().fit(x_kbest, y)

mean_squared_error(
    swiss.Fertility,
    model_kbest.predict(swiss[kbest_cols])
)

67.67925273589965

In [32]:
rfe_cols = rfe(x, y, 3)
x_rfe = x[rfe_cols]

model_rfe = LinearRegression().fit(x_rfe, y)

mean_squared_error(
    swiss.Fertility,
    model_rfe.predict(swiss[rfe_cols])
)

58.17259228615831